### Tensorflow NMT implementation: Will Cover 
- bi-direction GRU
- Attention Model
- Language Model
- Highway Networks (Probably)
- Tensorboard Functionalies

### TODO: Change this ipynb into a project

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from nmt_utils.helper import data_helper
from tqdm import tqdm

In [3]:
tf.reset_default_graph() #Clears the default graph stack and resets the global default graph.
sess = tf.InteractiveSession() #initializes a tensorflow session

### OPTIONS

In [7]:
class opt:
    #source_vocab_size = None,
    #dest_vocab_size  =None,
    source_embedding_size = 1000
    dest_embedding_size = 1000
    source_training_file = "/media/user/069A453E9A452B8D/Ram/projectsData/nmt/train.en"
    dest_training_file = "/media/user/069A453E9A452B8D/Ram/projectsData/nmt/train.vi"
    source_val_file = "/media/user/069A453E9A452B8D/Ram/projectsData/nmt/tst2012.en"
    dest_val_file = "/media/user/069A453E9A452B8D/Ram/projectsData/nmt/tst2012.vi"
    source_vocab_file = "/media/user/069A453E9A452B8D/Ram/projectsData/nmt/vocab.en"
    dest_vocab_file = "/media/user/069A453E9A452B8D/Ram/projectsData/nmt/vocab.vi"
    log_dir="tmp/ckpt1/"
    
#     source_max_length = None
#     dest_max_length = None    
#     source_vocab_size= None
#     dest_vocab_size=None
    batch_size = 4
#     source_vocab= None
#     dest_vocab = None
    
    #encoder_hidden_units = 2
    encoder_hiddent_units_size = [256,256]
    #decoder_hidden_units = 2
    decoder_hidden_units_size = [512, 256]
    
    share_vocab = False
    time_major = False
    encoder_state_to_decoder = False
    attention_depth = 256 ## attention Ai
    attention_layer_size = 128 ###
    
    mode = "train" ## or "infer"
    
    optimizer = "Adam" #or "SGD"
    nepochs = 10
    val_epoch  = 2 #every 2 epoch
    save_sess_epoch = 5 #every  
    
data = data_helper(opt)   

UNK = "<unk>"
SOS = "<s>"
EOS = "</s>"
UNK_ID = 0

#opt.source_vocab_file, 

### EMBEDDINGS

In [5]:
with tf.variable_scope('placeholders') as scope:
    encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
    encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

    decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')
    decoder_targets_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='decoder_targets_length')
    
    #source_start_token = tf.placeholder(shape=(1,), dtype=tf.int32, name='start_token')
    #source_end_token = tf.palceholder(shape=(1,), dtype=tf.int32, name='end_token')
    
    #dest_start_token = tf.placeholder(shape=(1,), dtype=tf.int32, name='start_token')
    #dest_end_token = tf.placeholder(shape=(1,), dtype=tf.int32, name='end_token')

In [6]:
# source_max_length = None
# dest_max_length = None    

# source_vocab_size= None
# dest_vocab_size=None

# batch_size = None

# source_vocab= None
# dest_vocab = None

source_max_length, _ = tf.unstack(tf.shape(encoder_inputs))
dest_max_length, _ = tf.unstack(tf.shape(decoder_targets))

source_vocab, source_vocab_size = data.load_vocab(opt.source_vocab_file)
dest_vocab, dest_vocab_size = data.load_vocab(opt.dest_vocab_file)

src_lookup_table, dest_lookup_table = data.create_vocab_tables()

dest_start_token = [tf.cast(dest_lookup_table.lookup(tf.constant(SOS)), tf.int32)]
dest_end_token = [tf.cast(dest_lookup_table.lookup(tf.constant(EOS)), tf.int32)]

In [7]:
with tf.variable_scope("embed") as scope:
    input_embeddings = tf.contrib.layers.embed_sequence(encoder_inputs, vocab_size = source_vocab_size,
                                embed_dim=opt.source_embedding_size)
    
    decoder_targets_embeddings = tf.contrib.layers.embed_sequence(decoder_targets, 
                                                                  vocab_size = dest_vocab_size,
                                                                  embed_dim = opt.dest_embedding_size, 
                                                                  scope = "decoder_embeddings")
    
    if opt.time_major:
        print("Time Major: True")
        input_embeddings = input_embeddings.swapaxes(0, 1) ## Change  Batch Major  to Time Major
        decoder_targets_embeddings = decoder_targets_embeddings.swapaxes(0, 1)

    
    with tf.variable_scope("decoder_embeddings", reuse = True):
        decoder_embeddings_mat = tf.get_variable("embeddings")


In [8]:
with tf.variable_scope('encoder') as scope:
    inputs_to_next = input_embeddings
    encoder_fw_cells = []
    encoder_bw_cells = []
    
    index = 0
    for units in opt.encoder_hiddent_units_size:
        with tf.variable_scope("encoder_fw_"+str(index)):
            encoder_fw_cells.append(tf.nn.rnn_cell.GRUCell(num_units = units)) # name = "encoder_fw_"+str(index) 
                                                                            # supported in tf1.6 maybe 1.5 also
        with tf.variable_scope("encoder_bw_"+str(index)):                                                                   
            encoder_bw_cells.append(tf.nn.rnn_cell.GRUCell(num_units = units)) # name = "encoder_bw_"+str(index)
        index = index + 1
    
    for index in range(len(opt.encoder_hiddent_units_size)):
        with tf.variable_scope("encoder_bidi_"+str(index)):
            ((encoder_fw_outputs,
                  encoder_bw_outputs),
             (encoder_fw_final_state,
                  encoder_bw_final_state)) = tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_fw_cells[index],
                                                        cell_bw=encoder_bw_cells[index],
                                                        inputs=inputs_to_next,
                                                        sequence_length=encoder_inputs_length,
                                                        dtype=tf.float32, time_major=opt.time_major,
                                                        scope = "encoder_bidi_rnn"+str(index))
        
        inputs_to_next = tf.concat((encoder_fw_outputs,encoder_bw_outputs),2)


    encoder_outputs = inputs_to_next
    encoder_final_state = tf.concat((encoder_fw_final_state, encoder_bw_final_state), 1)

### About GRUcells and sharing weights

name: String, the name of the layer. Layers with the same name will share weights, but to avoid mistakes we require reuse=True in such cases.

### For LSTM: encoder outputs will be      
     encoder_final_state_c = tf.concat(
                     (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

     encoder_final_state_h = tf.concat(
                     (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

     #TF Tuple used by LSTM Cells for state_size, zero_state, and output state.
     encoder_final_state = tf.nn.rnn_cell.LSTMStateTuple(
                                 c=encoder_final_state_c,
                                 h=encoder_final_state_h)

In [9]:
def decoder(helper, reuse = None):
    with tf.variable_scope("decoder", reuse = reuse) as scope:

        decoder_cells = []

        index  = 0
        for units in opt.decoder_hidden_units_size:
            with tf.variable_scope("decoder_cell_"+str(index)):
                decoder_cells.append(tf.nn.rnn_cell.GRUCell(num_units = units)) #name = "decoder_cell_"+str(index)
            index = index + 1

        decoder_stack = tf.nn.rnn_cell.MultiRNNCell(decoder_cells)

        attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units=opt.attention_depth, 
                                                                memory=encoder_outputs,
                                                                memory_sequence_length=None)  #TODO:  adjust memory_sequence_length 
        attn_cell = tf.contrib.seq2seq.AttentionWrapper(
                    decoder_stack, attention_mechanism, alignment_history = False,
                   attention_layer_size=opt.attention_layer_size) ##TODO: alignment_history = False

        out_cell = tf.contrib.rnn.OutputProjectionWrapper(
                    attn_cell, dest_vocab_size, reuse=reuse)

        decoder = tf.contrib.seq2seq.BasicDecoder(
                    out_cell, helper=helper,
                    initial_state=out_cell.zero_state(
                        dtype=tf.float32, batch_size=opt.batch_size))


        outputs = tf.contrib.seq2seq.dynamic_decode(
                    decoder=decoder, output_time_major=opt.time_major,
                    impute_finished=True, maximum_iterations=None)         #maximum_iterations=dest_max_length
        return outputs[0]    ##final_outputs, final_state, final_sequence_lengths
        
train_helper = tf.contrib.seq2seq.TrainingHelper(decoder_targets_embeddings, 
                                                 decoder_targets_length, time_major=opt.time_major)

#batch_size = tf.unstack(tf.shape()) ## TODO
start_tokens =  tf.fill(tf.cast([opt.batch_size], tf.int32), dest_start_token[0])
start_tokens = tf.Print(start_tokens, start_tokens, message="Start_Tokens: ", first_n=2)
infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings_mat, 
                                                        start_tokens=start_tokens, end_token=dest_end_token[0])

### DATA

In [10]:
# The encoder doesn't need GO & you can merge EOS & PAD (as long as sequence lengths are correct). 
# Here are an example with a batch of 2 sentences for training:
# (a) Encoder inputs (encoder lengths = [3, 2]):
# a b c EOS
# d e EOS EOS
# (b) Decoder inputs (decoder lengths = [4, 3]):
# GO 1 2 3
# GO 4 5 EOS
# (c) Decoder outputs (shift-by-1 of decoder inputs):
# 1 2 3 EOS
# 4 5 EOS EOS
# (the first EOS is part of the loss)

# During inference: we only have (a) + GO symbol fed to GreedyEmbeddingHelper, 
#so it's important to make sure this GO is the same as the GO in (b).


def get_src_tgt_datasaet(src_dataset,
                 dest_dataset,
                 src_lookup_table,
                 dest_lookup_table,
                 batch_size,
                 #src_max_len=None,
                 #dest_max_len=None,
                 num_parallel_calls = 4,
                 reshuffle_each_iteration=True,
                 buffer_size = None):
    
    if not buffer_size:
        buffer_size = batch_size*1000
        
    src_tgt_dataset = tf.data.Dataset.zip((src_dataset, dest_dataset))
    src_tgt_dataset = src_tgt_dataset.map(
            lambda src,dest: (tf.string_split([src]).values, tf.string_split([dest]).values),
            num_parallel_calls = num_parallel_calls
                ).prefetch(buffer_size)
    
    src_tgt_dataset = src_tgt_dataset.map(
            lambda src,dest:(tf.cast(src_lookup_table.lookup(src),tf.int32),
                             tf.cast(dest_lookup_table.lookup(dest),tf.int32)), 
            num_parallel_calls = num_parallel_calls
                ).prefetch(buffer_size)
    
    
    src_eos_id = tf.cast(src_lookup_table.lookup(tf.constant(EOS)), tf.int32)
    dest_eos_id = tf.cast(dest_lookup_table.lookup(tf.constant(EOS)), tf.int32)
    dest_sos_id = tf.cast(dest_lookup_table.lookup(tf.constant(SOS)), tf.int32)
    
    src_tgt_dataset = src_tgt_dataset.map(
            lambda src,dest:(tf.concat((src, [src_eos_id]), 0),
                            tf.concat(([dest_sos_id], dest, [dest_eos_id]), 0)), 
            num_parallel_calls = num_parallel_calls
                ).prefetch(buffer_size)
    
    src_tgt_dataset = src_tgt_dataset.map(
            lambda src,dest:( {'src':  src, 'dest': dest, 'src_len': [tf.size(src)], 'dest_len':[tf.size(dest)]}), 
            num_parallel_calls = num_parallel_calls
                ).prefetch(buffer_size)
    
    src_tgt_dataset = src_tgt_dataset.repeat(10)   ##TODO: for now repeat for 10 epochs 
    src_tgt_dataset = src_tgt_dataset.shuffle(buffer_size)
    src_tgt_dataset = src_tgt_dataset.padded_batch(batch_size, 
                                        padded_shapes={'src': [None], 'dest': [None], 'src_len':[None], 'dest_len':[None]},           
                                        padding_values = {'src': src_eos_id, 'dest': dest_eos_id, 'src_len':0, 'dest_len':0})
    
    return src_tgt_dataset
    

def get_reinitializable_iterator(src_tgt_dataset):
    return tf.data.Iterator.from_structure(src_tgt_dataset.output_types, src_tgt_dataset.output_shapes)

#training_init_op = iterator.make_initializer(training_dataset)
#validation_init_op = iterator.make_initializer(validation_dataset)


In [11]:
dest_eos_id = tf.cast(dest_lookup_table.lookup(tf.constant(EOS)), tf.int32)
dest_sos_id = tf.cast(dest_lookup_table.lookup(tf.constant(SOS)), tf.int32)

src_train_dataset = tf.data.TextLineDataset(opt.source_training_file)
dest_train_dataset = tf.data.TextLineDataset(opt.dest_training_file)

src_val_dataset  = tf.data.TextLineDataset(opt.source_val_file)
dest_val_dataset = tf.data.TextLineDataset(opt.dest_val_file)

training_dataset = get_src_tgt_datasaet(src_train_dataset, dest_train_dataset, src_lookup_table, dest_lookup_table, batch_size=opt.batch_size)
validation_dataset = get_src_tgt_datasaet(src_val_dataset, dest_val_dataset, src_lookup_table, dest_lookup_table, batch_size=opt.batch_size)

iterator = get_reinitializable_iterator(training_dataset)
next_element = next_element = iterator.get_next() 

training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

### TRAINING and VALIDATION

tf.sequence_mask([1, 3, 2], 5)  <br />
[[True, False, False, False, False], 
[True, True, True, False, False],
[True, True, False, False, False]]
                                
seq2seq_outputs = sess.run(training_decoder_outputs, feeding_dict) <br />
np.argmax(seq2seq_outputs.rnn_output[0][0]) = seq2seq_outputs.sample_id[0][0]                          

#### TensorBoard Logging

In [ ]:
training_decoder_outputs = decoder(train_helper)
infer_decoder_outputs = decoder(infer_helper, reuse=True)

with tf.variable_scope("loss"):
    targets_ = tf.gather(decoder_targets,  list(range(1,decoder_targets.shape[1])), axis=1)
    end_tokens =  tf.fill(tf.cast([opt.batch_size], tf.int32), dest_end_token[0])
    targets = tf.concat([targets_, tf.expand_dims(end_tokens, 1)], 1) 
    weights = tf.to_float(tf.sequence_mask(decoder_targets_length, decoder_targets[1]))
    loss_op = tf.contrib.seq2seq.sequence_loss(
                training_decoder_outputs.rnn_output, targets, weights)
    tf.summary.scalar("loss", loss_op)
    
with tf.variable_scope("accuracy"):
    with tf.variable_scope("train_acc"):
        train_correct_predictios = tf.equal(decoder_targets, training_decoder_outputs.sample_id)
        train_accuracy_op = tf.reduce_mean(train_correct_predictions)
        tf.summary.scalar("train_accuracy", train_accuracy_op)
    
    with tf.variable_scope("val_acc"):
        val_correct_predictions = tf.equal(decoder_targets, infer_decoder_outputs.sample_id)
        val_accuracy_op  = tf.reduce_mean(val_correct_predictions)
        tf.summary.scalar("val_accuracy", val_accuracy_op)

In [ ]:
with tf.variable_scope("train_op"):
    assert opt.optimizer == "Adam" or opt.optimizer == "SGD"
    if opt.optimizer == "Adam":
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08)
    elif opt.optimizer == "SGD":
        optimizer = tf.train.MomentumOptimizer(learning_rate = 0.001, momentum = 0.9,  use_nesterov= True)
    
    train_op = tf.contrib.layers.optimize_loss(
                loss, tf.train.get_global_step(),
                optimizer=optimizer
                learning_rate=None,
                summaries=['loss', 'learning_rate'],
                learning_rate_decay_fn=None,
        
                )

In [ ]:
writer = tf.summary.FileWriter(opt.log_dir)
writer.add_graph(sess.graph)

In [12]:
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

for epoch in tqdm(range(opt))
sess.run(training_init_op)

In [13]:
data_batch = sess.run(next_element)

In [15]:
feeding_dict = {
    encoder_inputs: data_batch['src'],
    encoder_inputs_length : data_batch['src_len'][:,0],
    decoder_targets: data_batch['dest'],
    decoder_targets_length: data_batch['dest_len'][:,0],
    #dest_start_token: np.array(dest_sos_id),
    #dest_end_token: np.array(dest_eos_id)
}

seq2seq_outputs = sess.run(training_decoder_outputs, feeding_dict)
# np.argmax(seq2seq_outputs.rnn_output[0][0]) = seq2seq_outputs.sample_id[0][0]

In [ ]:
#tf.sequence_mask([1, 3, 2], 5)  # [[True, False, False, False, False],
                                #  [True, True, True, False, False],
                                #  [True, True, False, False, False]]
        
# add  accuracy for validation

# add regularization in loss